# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import sys
sys.path.append('../../../')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""19694558908379…",1.2927e19,"""../../../../da…","""18443219898024…","""10000565921688…","""12401373531055…",false
"""34200049162623…",1.4856e19,"""../../../../da…","""16639811927336…","""10000565921688…","""54312388497045…",true
"""18192903196950…",5.6458e17,"""../../../../da…","""16551340893371…","""93848263936167…","""12401373531055…",false
"""34200049162623…",4.0723e18,"""../../../../da…","""15714897594549…","""60085797319458…","""12401373531055…",false
"""19694558908379…",1.4677e19,"""../../../../da…","""15360074740241…","""60085797319458…","""16623789450653…",true
"""18192903196950…",1.6330e19,"""../../../../da…","""95708445428108…","""14478502077818…",null,false
"""18192903196950…",8.7675e18,"""../../../../da…","""12313012254356…","""60085797319458…","""16623789450653…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_MaritalStatus': 'PREDICTOR_0',
 'Customer_CLV': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_City': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder='../../../../data/',
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""10094581563828…",1.2927e19,"""../../../../da…","""93052505995320…","""FirstMortgage3…","""48042678639708…","""Rejected"""
"""40305442818521…",1.4856e19,"""../../../../da…","""10305082677968…","""FirstMortgage3…","""97225936046728…","""Accepted"""
"""10039905418250…",5.6458e17,"""../../../../da…","""62002066751993…","""MoneyMarketSav…","""48042678639708…","""Rejected"""
"""40305442818521…",4.0723e18,"""../../../../da…","""10538800893059…","""BasicChecking""","""48042678639708…","""Rejected"""
"""10094581563828…",1.4677e19,"""../../../../da…","""68156763558680…","""BasicChecking""","""16036067014078…","""Accepted"""
"""10039905418250…",1.6330e19,"""../../../../da…","""17779605866883…","""UPlusFinPerson…",null,"""Rejected"""
"""10039905418250…",8.7675e18,"""../../../../da…","""13761849274662…","""BasicChecking""","""16036067014078…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""16656280198478…",1.2927e19,"""../../../../da…","""32927789816399…","""FirstMortgage3…","""13254902363640…","""Rejected"""
"""80341693435348…",1.4856e19,"""../../../../da…","""18297710567298…","""FirstMortgage3…","""14739927571452…","""Accepted"""
"""11541417582554…",5.6458e17,"""../../../../da…","""38330429152660…","""MoneyMarketSav…","""13254902363640…","""Rejected"""
"""80341693435348…",4.0723e18,"""../../../../da…","""20243887099162…","""BasicChecking""","""13254902363640…","""Rejected"""
"""16656280198478…",1.4677e19,"""../../../../da…","""12289417529554…","""BasicChecking""","""16621756495398…","""Accepted"""
"""11541417582554…",1.6330e19,"""../../../../da…","""13628448142531…","""UPlusFinPerson…",null,"""Rejected"""
"""11541417582554…",8.7675e18,"""../../../../da…","""13624537387367…","""BasicChecking""","""16621756495398…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_MaritalStatus=PREDICTOR_0
Customer_CLV=PREDICTOR_1
filename=filename
Customer_City=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""56715673773783…",1.2927e19,"""19409979304041…","""FirstMortgage3…","""82143594820648…","""Rejected"""
"""12047831192238…",1.4856e19,"""22347813483447…","""FirstMortgage3…","""95923165031171…","""Accepted"""
"""60162546753240…",5.6458e17,"""33166164906163…","""MoneyMarketSav…","""82143594820648…","""Rejected"""
"""12047831192238…",4.0723e18,"""17101042543073…","""BasicChecking""","""82143594820648…","""Rejected"""
"""56715673773783…",1.4677e19,"""18385024727712…","""BasicChecking""","""15338702086004…","""Accepted"""
"""60162546753240…",1.6330e19,"""97961411622136…","""UPlusFinPerson…",null,"""Rejected"""
"""60162546753240…",8.7675e18,"""15031701167194…","""BasicChecking""","""15338702086004…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'